In [1]:
import albumentations as A


from dataset import *

import numpy as np

import torch
import torchvision.models as models


from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from utils import *
from code_eval import code

import scipy.io as sio

import torch.nn as nn

from torchvision.models import EfficientNet_B3_Weights

size_img = 300


data_transforms_albumentation = {
    'train': A.Compose([
        A.ElasticTransform(),
        A.GridDistortion(),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(size_img, size_img),
        ToTensorV2(),
    ]),
    'test': A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(size_img, size_img),
        ToTensorV2(),
    ])
}



In [ ]:

torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

print("Loading datasets")

train = sio.loadmat("train_data.mat")
test = sio.loadmat("test_data.mat")



train_images = train["images"]


irma_train = np.vstack(train["irma_code"][0])[:,0]


label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(irma_train)


evaluator = code("codes.txt")

model = models.efficientnet_b3(weights = EfficientNet_B3_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = True
model.classifier = nn.Sequential(nn.Dropout(
    0.2), nn.Linear(in_features=1536, out_features=193))
model.to(device)

sampler = get_sampler(train_labels)

train_dataset = ImagesDataset(
    train_images, train_labels, irma_train, transform=data_transforms_albumentation["train"], albumentation=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=8, num_workers=0, sampler=sampler)


optimizer = optim.SGD(model.parameters(), lr=0.01,  weight_decay=0.000001)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=15,  gamma=0.5, verbose=True)

criterion = nn.CrossEntropyLoss()

for i in (range(100)):
    print(f"Epoch {i}")
    model = model.train()
    _, _, model = fit_epoch(model, optimizer, criterion,
                            train_loader, device, verbose=True)
    scheduler.step()
    model.eval()
    torch.save(model.state_dict(), f".\\model_final.pth")
